In [ ]:
from tqdm.notebook import tqdm

In [ ]:
"""
Purpose: To pull down the earliest version of thhe synapses from 11/9

"""

In [3]:
from annotationframeworkclient import FrameworkClient
from IPython import html, display

/usr/local/lib/python3.6/dist-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [4]:
client = FrameworkClient('minnie65_phase3_v1')

# Getting the synapses for the neurons we have 

In [5]:
from os import sys
sys.path.append("/meshAfterParty")

In [6]:
import datajoint_utils as du

In [7]:
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-25 17:54:44,789 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-25 17:54:44,790 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-25 17:54:44,804 - connection - Connected celiib@at-database.ad.bcm.edu:3306


Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-25 17:54:45,020 - settings - Setting enable_python_native_blobs to True


In [8]:
import numpy as np
soma_segment_ids = np.unique((minnie.BaylorSegmentCentroid & "multiplicity>0").fetch("segment_id"))

In [9]:
import pandas as pd

def fix_dataframe_and_return_dict(df):
    if len(df) == 0:
        return []
    df[["centroid_x","centroid_y","centroid_z"]] = pd.DataFrame(df["ctr_pt_position"].tolist(),index=df.index)
    df_renamed = df.rename(columns=dict(pre_pt_root_id="presyn",post_pt_root_id="postsyn",id="synapse_id"))
    B = df_renamed.drop(["valid","pre_pt_position","ctr_pt_position","post_pt_position"],axis=1)
    B = B.astype({'size': "int32"})
    #print(B.dtypes)
    return B.to_dict("records")



verbose = False

step_size = 10
max_return_size = 200000
total_ids = soma_segment_ids
n_iterations = int(np.ceil(len(total_ids)/step_size))
n_jobs = 6
iterations_per_job = n_iterations/n_jobs
iterations_per_job

1231.3333333333333

In [10]:
"""
Pseudocode: 

in batches of 100 segment ids

for both presyn and postsyn:
1) Get the rows for that attirbute and those segments and turn into dictionaries and add to list
2a) If the result is less than 200k don't need to repeat again 
2b) If they are greater, then repeat 1 and 2 (and add offset) until less than 200k
3) Write the entries to datajoint

"""

job_idx = 5


for i in tqdm(range(int(iterations_per_job*job_idx)+117,n_iterations)):
    if i == (n_iterations - 1):
        curr_seg_ids =  total_ids[i*step_size:]
    else:
        curr_seg_ids = total_ids[i*step_size:(i+1)*step_size]
        
    total_dicts = []
    for filter_type in ["pre_pt_root_id","post_pt_root_id"]:
        curr_filter_dict = {filter_type:curr_seg_ids}
        curr_offset = 0
        #print(f"curr_filter_dict = {curr_filter_dict}")
        while True:
            df = client.materialize.query_table('synapses_pni_2',
                                                #filter_in_dict=dict(func_id=[58941,16062]),
                                                filter_in_dict=curr_filter_dict, #just excludes those

                                                #filter_equal_dict=dict(func_id=58941),
                                               offset=curr_offset ,#this is an integer that will set the offset
                                                #select_columns=columns_we_want
                                               )
            
            df_dicts = fix_dataframe_and_return_dict(df)
            total_dicts += df_dicts
            if len(df_dicts) < max_return_size:
                break
            else:
                curr_offset += max_return_size
    
    #write the data to the datajoint table
    if len(total_dicts) > 0:
        if verbose:
            print(f"Inserting {len(total_dicts)} dictionaries ")
        minnie.SynapseTest.insert(total_dicts,skip_duplicates=True)
    else:
        if verbose:
            print("Skipping dictionary insertion because it was empty")
            
    
    #raise Exception("")
            

OperationalError: (1213, 'Deadlock found when trying to get lock; try restarting transaction')

# Datajoint table to dump the results into 